In [ ]:
#Set-up code
!pip install pandas
!pip install scipy
!pip install sklearn
!pip install openpyxl
!pip install numpy
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandas.core.arrays import categorical
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.metrics import Precision, Recall, BinaryAccuracy
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn import preprocessing
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

from scipy.stats import zscore
import matplotlib.pyplot as plt

In [ ]:
# Load the datasets
ratings = pd.read_csv('sample_data/amazon_50k.csv')
empty_cols = [col for col in ratings.columns if ratings[col].isnull().all()]

if len(empty_cols) > 0:
    # Remove the empty column
    ratings = ratings.drop(columns=empty_cols)

ratings.columns = ['userId', 'itemId', 'rating']
print(ratings)




               userId      itemId  rating
0      A2OXDRWBASV91Y  B00004SQHD       5
1      A2KG6AWJSWILPR  B00004SQHD       5
2      A2CBE6VYOARZN4  B00004SQHD       5
3       AVKOTZD5ZIOX5  B00004SQHD       5
4      A33DUVUIC7G553  B00004SQHD       5
...               ...         ...     ...
53764   A1JCY9IVF34DE  B000THKV4O       1
53765  A3B834ZT5C1GCW  B000THKV4O       5
53766  A23NSKTMSPPBTR  B000THKV4O       5
53767  A2KP45DO3RY4RG  B000THKV4O       5
53768   A9YKOX54Y35KL  B000THKV4O       5

[53769 rows x 3 columns]


In [ ]:
# Create a binary column indicating whether a rating is suspicious (1) or not (0)
#ratings['suspicious'] = np.where(ratings['rating'] >= 4, 0, 1)
#print(ratings)

mean = np.mean(ratings['rating'])
std = np.std(ratings['rating'])
ratings['z-score'] = (ratings['rating'] - mean)/std

threshold = 2.0
ratings['sus'] = np.where(ratings['z-score'].abs() > threshold, 1, 0)
print(ratings)


user_encoder = preprocessing.LabelEncoder()
user_encoder.fit(ratings['userId'])
item_encoder = preprocessing.LabelEncoder()
ratings['userId'] = user_encoder.transform(ratings['userId'])
ratings['itemId'] = item_encoder.fit_transform(ratings['itemId'])
print('------------------encoded user and item IDs---------------------')
print(ratings)



               userId      itemId  rating   z-score  sus
0      A2OXDRWBASV91Y  B00004SQHD       5  0.595943    0
1      A2KG6AWJSWILPR  B00004SQHD       5  0.595943    0
2      A2CBE6VYOARZN4  B00004SQHD       5  0.595943    0
3       AVKOTZD5ZIOX5  B00004SQHD       5  0.595943    0
4      A33DUVUIC7G553  B00004SQHD       5  0.595943    0
...               ...         ...     ...       ...  ...
53764   A1JCY9IVF34DE  B000THKV4O       1 -2.641896    1
53765  A3B834ZT5C1GCW  B000THKV4O       5  0.595943    0
53766  A23NSKTMSPPBTR  B000THKV4O       5  0.595943    0
53767  A2KP45DO3RY4RG  B000THKV4O       5  0.595943    0
53768   A9YKOX54Y35KL  B000THKV4O       5  0.595943    0

[53769 rows x 5 columns]
------------------encoded user and item IDs---------------------
       userId  itemId  rating   z-score  sus
0       23244       0       5  0.595943    0
1       21528       0       5  0.595943    0
2       18428       0       5  0.595943    0
3       50141       0       5  0.595943    0


In [ ]:
# Split the dataset into training and testing sets
train, test = train_test_split(ratings, test_size=0.2, random_state=42)

# Convert the training and testing sets into arrays
X_train = np.array(train['userId'])
y_train = np.array(train['sus'])
X_test = np.array(test['userId'])
y_test = np.array(test['sus'])

# Compute the number of unique user IDs in the dataset
num_users = ratings['userId'].nunique()

# Convert the labels to one-hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Reshape the input data to be a 3D tensor with shape (samples, time steps, features)
X_train = np.reshape(X_train, (X_train.shape[0], 1, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 1, 1))

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, 1)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.RootMeanSquaredError(), f1_m, tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

# fit the model
history = model.fit(X_train, y_train, validation_split=0.3, epochs=10, verbose=0)

#Make predictions on the train and test sets
y_train_pred = model.predict(X_train)
y_train_pred = (y_train_pred > 0.5).astype(int)
y_test_pred = model.predict(X_test)
y_test_pred = (y_test_pred > 0.5).astype(int)

#Generate true and predicted labels
y_train_true = y_train.flatten()
y_test_true = y_test.flatten()

# evaluate the model
loss, rmse, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)

#print the results
print("MSE: ", loss)
print("RMSE: ", rmse)
print("F1 Score: ", f1_score)
print("Precision: ", precision)
print("Recall: ",  recall)

337/337 [==============================] - 1s 2ms/step
MSE:  0.07643666863441467
RMSE:  0.2764718234539032
F1 Score:  0.9237759709358215
Precision:  0.9235633015632629
Recall:  0.9235633015632629


In [ ]:
#DO NOT RUN FOR NOW
#Todo: Update the plot to show calculated metrics


# Plot training & validation accuracy values
plt.figure(figsize=(10,6))
plt.plot(history['accuracy'], '*-')
plt.plot(history['val_accuracy'], '*-')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()
plt.close()


# Plot training & validation loss values
plt.figure(figsize=(10,6))
plt.plot(history['loss'], '*-')
plt.plot(history['val_loss'], '*-')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.tight_layout()
plt.show()
plt.close()


TypeError: ignored

<Figure size 720x432 with 0 Axes>

In [ ]:
#load infected dataset
ratings_infected = pd.read_csv('sample_data/amazon_50k_bandwagon.csv')
empty_cols = [col for col in ratings_infected.columns if ratings_infected[col].isnull().all()]

if len(empty_cols) > 0:
    # Remove the empty column
    ratings_infected = ratings_infected.drop(columns=empty_cols)

ratings_infected.columns = ['userId', 'itemId', 'rating']
print(ratings_infected)



mean_infected = np.mean(ratings_infected['rating'])
std_infected = np.std(ratings_infected['rating'])
ratings_infected['z-score'] = (ratings_infected['rating'] - mean_infected)/std_infected

threshold = 2.0
ratings_infected['sus'] = np.where(ratings_infected['z-score'].abs() > threshold, 1, 0)
print(ratings_infected)

user_encoder_i = preprocessing.LabelEncoder()
user_encoder_i.fit(ratings_infected['userId'])
item_encoder_i = preprocessing.LabelEncoder()
ratings_infected['userId'] = user_encoder_i.transform(ratings_infected['userId'])
ratings_infected['itemId'] = item_encoder_i.fit_transform(ratings_infected['itemId'])
print('------------------encoded user and item IDs---------------------')
print(ratings_infected)

# Split the dataset into training and testing sets
train_infected, test_infected = train_test_split(ratings_infected, test_size=0.2, random_state=42)

# Convert the training and testing sets into arrays
X_train_infected = np.array(train_infected['userId'])
y_train_infected = np.array(train_infected['sus'])
X_test_infected = np.array(test_infected['userId'])
y_test_infected = np.array(test_infected['sus'])

# Compute the number of unique user IDs in the dataset
num_users_infected = ratings_infected['userId'].nunique()

# Convert the labels to one-hot encoding
y_train_infected = to_categorical(y_train_infected)
y_test_infected = to_categorical(y_test_infected)

# Reshape the input data to be a 3D tensor with shape (samples, time steps, features)
X_train_infected = np.reshape(X_train_infected, (X_train_infected.shape[0], 1, 1))
X_test_infected = np.reshape(X_test_infected, (X_test_infected.shape[0], 1, 1))

#Make predictions on the train and test sets
y_train_pred_i = model.predict(X_train_infected)
y_train_pred_i = (y_train_pred_i > 0.5).astype(int)
y_test_pred_i = model.predict(X_test_infected)
y_test_pred_i = (y_test_pred_i > 0.5).astype(int)

# evaluate the model
loss_i, rmse_i, f1_score_i, precision_i, recall_i = model.evaluate(X_test, y_test, verbose=0)

#print the results
print("MSE: ", loss_i)
print("RMSE: ", rmse_i)
print("F1 Score: ", f1_score_i)
print("Precision: ", precision_i)
print("Recall: ",  recall_i)

               userId      itemId  rating
0      A3C4IQ321NXOYM  B0009GVYNW       5
1      A2L9O4TZT4IJG6  B0000CFPK8       5
2      A3QEWQ4XHNY6U0  B000GX34AE       1
3       AJWGJBKAPT1B4  B000FNKV2Y       5
4      A25ZR0LKZPUYIH  B000AST3AK       4
...               ...         ...     ...
62300           52058  B000OSLUMK       4
62301           52058  B0001ZLVCK       4
62302           52058  B000HEDV5A       4
62303           52058  B0009JLRNG       4
62304           52058  B0002D000K       4

[62305 rows x 3 columns]
               userId      itemId  rating   z-score  sus
0      A3C4IQ321NXOYM  B0009GVYNW       5  0.672579    0
1      A2L9O4TZT4IJG6  B0000CFPK8       5  0.672579    0
2      A3QEWQ4XHNY6U0  B000GX34AE       1 -2.672466    1
3       AJWGJBKAPT1B4  B000FNKV2Y       5  0.672579    0
4      A25ZR0LKZPUYIH  B000AST3AK       4 -0.163682    0
...               ...         ...     ...       ...  ...
62300           52058  B000OSLUMK       4 -0.163682    0
62301         

In [ ]:
tf.compat.v1.reset_default_graph()